In [13]:
# 导入所需的包
import tushare as ts
import csv
import time
import pandas as pd
pro = ts.pro_api('4340a981b3102106757287c11833fc14e310c4bacf8275f067c9b82d')

### 1. 金融数据获取

#### 1.1 股票基本信息

In [2]:
# 获取股票基本信息
stock_basic = pro.stock_basic(list_status='L', fields='ts_code, symbol, name, industry')
# 重命名行，便于后面导入neo4j
basic_rename = {'ts_code': 'TS代码', 'symbol': '股票代码', 'name': '股票名称', 'industry': '行业'}
stock_basic.rename(columns=basic_rename, inplace=True)
# 保存为stock_basic.csv
stock_basic.to_csv('financial_data\\stock_basic.csv', encoding='gbk')
stock_basic.head()

,TS代码,股票代码,股票名称,行业
0,000001.SZ,000001,平安银行,银行
1,000002.SZ,000002,万科A,全国地产
2,000004.SZ,000004,国农科技,生物制药
3,000005.SZ,000005,世纪星源,环境保护
4,000006.SZ,000006,深振业A,区域地产


#### 1.2 股票Top10股东信息

In [4]:
# 注意：该块代码运行缓慢
# 获取top10_holders
holders = pd.DataFrame(columns=('ts_code', 'ann_date', 'end_date', 'holder_name', 'hold_amount', 'hold_ratio'))
# 获取一年内所有上市股票股东信息（可以获取一个报告期的）
for i in range(3610):
    code = stock_basic['TS代码'].values[i]
    top10_holders = pro.top10_holders(ts_code=code, start_date='20180101', end_date='20181231')
    holders = holders.append(top10_holders)
    if i % 600 == 0:
        print(i)
    time.sleep(0.4)# 数据接口限制
# 保存为stock_holders.csv
holders.to_csv('financial_data\\stock_holders.csv', encoding='gbk')

top10_holders = pro.top10_holders(ts_code='000001.SZ', start_date='20180101', end_date='20181231')
top10_holders.head()

,ts_code,ann_date,end_date,holder_name,hold_amount,hold_ratio
0,000001.SZ,20190307,20181231,新华人寿保险股份有限公司-分红-个人分红-018L-FH002深,4.960350e+07,0.29
1,000001.SZ,20190307,20181231,中国平安保险(集团)股份有限公司-集团本级-自有资金,8.510493e+09,49.56
2,000001.SZ,20190307,20181231,中国平安人寿保险股份有限公司-自有资金,1.049463e+09,6.11
3,000001.SZ,20190307,20181231,香港中央结算有限公司(陆股通),4.307515e+08,2.51
4,000001.SZ,20190307,20181231,中国证券金融股份有限公司,4.292327e+08,2.50


#### 1.3 股票概念信息

In [5]:
# 获取concept_detail
concept_details = pd.DataFrame(columns=('id', 'concept_name', 'ts_code', 'name'))
for i in range(358):
    id = 'TS' + str(i)
    concept_detail = pro.concept_detail(id=id)
    concept_details = concept_details.append(concept_detail)
    time.sleep(0.4)
# 保存为concept_detail.csv
concept_details.to_csv('financial_data\\stock_concept.csv', encoding='gbk')
concept_detail.head()

,id,concept_name,ts_code,name
0,TS0,密集调研,000917.SZ,电广传媒
1,TS0,密集调研,002123.SZ,梦网集团
2,TS0,密集调研,002127.SZ,南极电商
3,TS0,密集调研,002371.SZ,北方华创
4,TS0,密集调研,002460.SZ,赣锋锂业


#### 1.4 股票公告信息

In [17]:
# 获取notices_detail
for i in range(3610):
    code = stock_basic['TS代码'].values[i]
    notices = pro.anns(ts_code=code, start_date='20180101', end_date='20181231', year='2018')
    notices.to_csv("financial_data\\notices\\"+str(code)+".csv",encoding='utf_8_sig',index=False)
notices = pro.anns(ts_code='000001.SZ', start_date='20180101', end_date='20181231', year='2018')
notices.head()

,ts_code,ann_date,title,content
0,000001.SZ,20181227,关于公开发行A股可转换公司债券申请获得中国证监会核准批复的公告,证券代码：000001 证券简称：平安银行 ...
1,000001.SZ,20181226,董事会决议公告,证券代码：000001 证券简称：平安银行 ...
2,000001.SZ,20181219,关于成功发行350亿元金融债券的公告,证券代码：000001 证券简称：平安银行 ...
3,000001.SZ,20181218,关于公开发行A股可转换公司债券申请获中国证监会发行审核委员会审核通过的公告,证券代码：000001 证券简称：平安银行 ...
4,000001.SZ,20181208,关于发行金融债券获准的公告,证券代码：000001 证券简称：平安银行 ...


#### 1.5 财经新闻信息

In [19]:
# 获取news
news = pro.news(src='sina', start_date='20180101', end_date='20181231')
news.to_csv("financial_data\\news.csv",encoding='utf_8_sig')
news.head()

,datetime,content,title
0,2018-12-30 23:44:21,意大利议会批准2019年的预算法。,
1,2018-12-30 23:38:45,【中央广播电视总台2018年国际十大新闻】1、中国成功举办全球首个进口博览会，推动开放型世界...,
2,2018-12-30 23:22:01,【超2亿份国内简历数据遭泄露】近日创宇盾网站安全舆情监测平台发现，某Twitter账户发布了...,
3,2018-12-30 22:33:32,德国安联首席经济学家El-Erian：美国正面临经济衰退的说法并非现实。美联储意识到不能让政...,
4,2018-12-30 22:18:09,美国纽约州州长Cuomo：受极端风暴影响的纽约州民众可以获得为期6个月的中期按揭贷款援助计划延期。,


#### 1.6 概念信息

In [20]:
# 获取概念信息
concept = pro.concept()
concept.to_csv('financial_data\\concept.csv', encoding='gbk')
concept.head()

,code,name,src
0,TS0,密集调研,ts
1,TS1,南北船合并,ts
2,TS2,5G,ts
3,TS3,机场,ts
4,TS4,高价股,ts


#### 1.7 沪股通成分和深股通成分信息 

In [22]:
#获取沪股通成分
sh = pro.hs_const(hs_type='SH') 
sh.to_csv("financial_data\\sh.csv",index=False)
#获取深股通成分
sz = pro.hs_const(hs_type='SZ')
sz.to_csv("financial_data\\sz.csv",index=False)
sh.head()

,ts_code,hs_type,in_date,out_date,is_new
0,603818.SH,SH,20160613,None,1
1,603108.SH,SH,20161212,None,1
2,600507.SH,SH,20141117,None,1
3,601377.SH,SH,20141117,None,1
4,600309.SH,SH,20141117,None,1


#### 1.8 股票价格信息

In [25]:
for i in range(3610):
    code = stock_basic['TS代码'].values[i]
    price = pro.query('daily', ts_code=code, start_date='20180101', end_date='20181231')
    price.to_csv("financial_data\\price\\"+str(code)+".csv",index=False)

#### 价格走势图 Example

In [90]:
from pyecharts.charts import Line
from pyecharts import options as opts
import numpy as np
price = pro.query('daily', ts_code='000001.SZ', start_date='20180101', end_date='20181231')
(
    Line()
    .add_xaxis(xaxis_data=list(price['trade_date'])[::-1])
    
    .add_yaxis(series_name="close price",y_axis=list(price['close'])[::-1],symbol="circle")
    .add_yaxis(series_name="open price",y_axis=list(price['open'])[::-1],symbol="circle")
    .add_yaxis(series_name="high price",y_axis=list(price['high'])[::-1],symbol="circle")
    .add_yaxis(series_name="low price",y_axis=list(price['low'])[::-1],symbol="circle")
    .set_global_opts(title_opts=opts.TitleOpts(title="价格走势图"))
    .render_notebook()
)